In [2]:
import pandas as pd
import numpy as np
import re
import datetime as dt
import pytz
import sys,os
ENV_PATH = '../../ENV/'
sys.path.append(ENV_PATH)
from env import ENV

__file__ = '../../classifier/models/time_pattern/'

class TimePattern:
    def __init__(self,tz=None):
        """
        tz = "Asia/Shanghai"
        tz = pytz.timezone(tz)
        'America/New_York'
        """
        self.pattern_csv = os.path.join(os.path.dirname(__file__), 'mapping.csv')
        self._set_timeZone(tz)
        self._load_mapping(self.pattern_csv)
        self.time_dic = {'今':'?','明':'+1','后':'+2','大后':'+3','下个':'+1','下下个':'+2',
                         '再下个':'+2','下下下个':'+3','后1个':'+1','后2个':'+2','后一个':'+1',
                         '后两个':'+2',
                         '一':'1','二':'2','三':'3','四':'4','五':'5',
                         '六':'6','七':'7','八':'8','九':'9','十':'10',
                         '十一':'11','十二':'12','十三':'13','十四':'14','十五':'15',
                         '十六':'16','十七':'17','十八':'18','十九':'19','二十':'20',
                         '二十一':'21','二十二':'22','二十三':'23','二十四':'24','二十五':'25',
                         '二十六':'26','二十七':'27','二十八':'28','二十九':'29','三十':'30','三十一':'31',
                         '四十':'40','五十':'50','六十':'60','七十':'70','八十':'80','九十':'90','一百':'100'}
        self.fix_ymd = r'(?:(?:今|明|后|大后)年)?(?:(?:\d{1,2}|下下下个|下下个|再下个|下个|十一|十二|一|二|三|四|五|六|七|八|九|十|后1个|后2个|后一个|后两个|后二个)月)(?:\d{1,2}|一|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四||十六|十七|十八|十九|二十|二十一|二十二|二十三|二十四|二十五|二十六|二十七|二十八|二十九|三十|三十一)[日号]'
        reg_num = r'(?:(?:一|二|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|二十|三十|四十|五十|六十|七十|八十|九十)|\d+)'
        self.fix_period = r'(?:{}天)?(?:{}小时)?(?:{}分钟)?'.format(reg_num,reg_num,reg_num)
        
    def remove_time(self,sentence):
        sentence = re.sub(r' ','',sentence)
        sentence = re.sub(self.re_ext,' ',sentence)
        
        finds_ymd = set(re.findall(self.fix_ymd,sentence)) - set([''])
        finds_period = set(re.findall(self.fix_period,sentence)) - set([''])
        if len(finds_ymd) > 0:
            sentence = re.sub('|'.join(finds_ymd),' ',sentence)
        if len(finds_period) > 0:
            sentence = re.sub('|'.join(finds_period),' ',sentence)

        if sentence == '':
            return ' '
        return sentence
        
    
    def process(self, sentence):
        current = self._get_LocalNow()
        sentence = re.sub(r" ",'',sentence)
        fixymd = self.evl_ymd(sentence)
        selfdefine = re.findall(self.re_ext, sentence)
        period = self.evl_period(sentence)
        result = []
        for each in fixymd:
            future = self.evl(each['expression'])
            gap = (future - current).total_seconds()
            result.append({'pattern':each['pattern'], 'time':future, 'gapS':gap, 'gapH':gap/3600})
        for each in selfdefine:
            future = self.evl(self.dict_ext[each])
            gap = (future - current).total_seconds()
            result.append({'pattern':each, 'time':future, 'gapS':gap, 'gapH':gap/3600})
        for each in period:
            future = self.evl(each['expression'])
            gap = (future - current).total_seconds()
            result.append({'pattern':each['pattern'], 'time':future, 'gapS':gap, 'gapH':gap/3600})
        return result
    
    def evl(self, expression):
        current = self._get_LocalNow()
        exp_week = re.findall(r'-.+W-.+w',expression)
        exp_ymd = re.findall(r'.+y-.+m.+d',expression)

        history = self._pros_second(expression, current)
        history = self._pros_minute(expression, current, history)
        history,shift = self._pros_hour(expression, current, history)

        if exp_week:
            history = self._pros_weekDay(expression,current, history)
            history = self._pros_week(expression,current, history)
            history = self._pros_year(expression, current,history)
            future = self.create_from_W(history)  
        elif exp_ymd:
            history = self._pros_day(expression, current,history)
            history = self._pros_month(expression, current,history)
            history = self._pros_year(expression, current,history)
            future = self.create_from_D(history)
        if future.tzinfo is None:
            future = self.tz.localize(future)
        return future
    
    def _load_mapping(self, pattern_path):
        df = pd.read_csv(pattern_path)
        # create length
        df['length'] = df.key_word.apply(lambda x: len(x))
        df = df.sort_values(['length','key_word'], ascending=False)
        df_series = pd.Series(index=df.key_word.values, data=df.expression.values)
        df_dict = df_series.to_dict()
        self.serires = df_series
        self.re_ext = r'|'.join(self.serires.index.values)
        self.dict_ext = df_dict
        
    def _set_timeZone(self,tz=None):
        if tz is None:
            tz = ENV.TIMEZONE.value
            print('Time Zone is set from ENV: {}'.format(tz))
        tz = ENV.TIMEZONE.value
        self.tz = pytz.timezone(tz)
        self.delta = self.tz.utcoffset(dt.datetime.utcnow())
        
        
    def _get_LocalNow(self):
        now = dt.datetime.utcnow()
        return self.tz.localize(now) + self.delta
        
    def _pros_second(self, expression, current, history={'microsecond':0}):
        history = history.copy()
        S = current.second
        reexp = r'M:.+S'
        extract = re.findall(reexp,expression)[0]
        # M:+1S
        if extract[2:-1] == '?':
            history.update({'second':S})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(seconds=gap) 
            create = create.replace(**history)
            return create 
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create =  current - dt.timedelta(seconds=gap)
            create = create.replace(**history)
            return create
        else:
            second = int(extract[2:-1])
            history.update({'second':second})
            return history
        
    def _pros_minute(self, expression, current, history = {}):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        M = current.minute
        reexp = r'H:.+M'
        extract = re.findall(reexp,expression)[0]
        # H:?M
        if extract[2:-1] == '?':
            history.update({'minute':M})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(minutes=gap) 
            create = create.replace(**history)
            return create
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(minutes=gap) 
            create = create.replace(**history)
            return create
        else:
            minute = int(extract[2:-1])
            history.update({'minute':minute})
            return history
        
    def _pros_hour(self, expression, current, history = {}):
        shift = True
        if isinstance(history,dt.datetime):
            return history, shift
        history = history.copy()

        H = current.hour
        reexp = r'[dw]-.+H'
        extract = re.findall(reexp,expression)[0]
    #     d-?H
        if extract[2:-1] == '?':
            history.update({'hour':H})
            return history,shift
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(hours=gap)
            create = create.replace(**history)
            return create,shift
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(hours=gap)
            create = create.replace(**history)
            return create,shift
        else:
            shift = False
            hour = int(extract[2:-1])
            history.update({'hour':hour})
            return history,shift
        
    def _pros_day(self, expression, current, history={}):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        d = current.day
        reexp = r'm-.+d'
        extract = re.findall(reexp,expression)[0]
        if extract[2:-1] == '?':
            history.update({'day':d})
            return history
        elif extract[2] == '+':
            gap = int(extract[3:-1])
            create = current + dt.timedelta(days=gap) 
            create = create.replace(**history)
            return create
        elif extract[2] == '-':
            gap = int(extract[3:-1])
            create = current - dt.timedelta(days=gap) 
            create = create.replace(**history)
            return create
        else:
            day = int(extract[2:-1])
            history.update({'day':day})
            return history
        
    def _pros_month(self, expression, current, history):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        adjust_year = 0
        m = current.month
        reexp = r'y-.+m'
        extract = re.findall(reexp,expression)[0]
        if extract[2:-1] == '?':
            history.update({'month':m})
            return history
        elif extract[2] == '+':
            cur = int(extract[3:-1]) + m
            if cur > 12:
                adjust_year = int(cur / 12)
                cur = cur % 12
                if cur == 0:
                    cur = 12
                    adjust_year -= 1
            history.update({'year':adjust_year})
            history.update({'month':cur})
            return history
        elif extract[2] == '-':
            cur = m - int(extract[3:-1])
            if cur < 1:
                adjust_year = int(cur / 12) - 1
                cur = cur % 12
                if cur == 0:
                    cur = 12
            history.update({'year':adjust_year})
            history.update({'month':cur})
            return history
        else:
            history.update({'month':int(extract[2:-1])})
            return history
        
    def _pros_year(self, expression, current, history):
        if isinstance(history,dt.datetime):
            return history
        history = history.copy()
        adjust_year = history.get('year')
        if adjust_year is None:
            adjust_year = 0
        y = current.year
        reexp = r'.+y-'
        extract = re.findall(reexp,expression)[0]
        if extract[0:-2] == '?':
            history.update({'year':y+adjust_year})
            return history
        elif extract[0] == '+':
            gap = int(extract[1:-2])
            history.update({'year':y+adjust_year+gap})
            return history
        elif extract[0] == '-':
            gap = int(extract[1:-2])
            history.update({'year':y+adjust_year-gap})
            return history
        else:
            history.update({'year':int(extract[:-2])+adjust_year})
            return history
        
    def _pros_weekDay(self, expression, current, history):
        history = history.copy()
        w = current.isocalendar()[2] % 7
        reexp = r'W-.+w'
        extract = re.findall(reexp,expression)[0]

        # W-+1w
        if extract[2:-1] == '?':
            history.update({'weekday':str(w)})
            return history
        elif extract[2] == '+':
            rep = str(w + int(extract[3:-1]))
            history.update({'weekday':rep})
            return history
        elif extract[2] == '-':
            rep = str(w - int(extract[3:-1]))
            history.update({'weekday':rep})
            return history
        else:
            rep = extract[2:-1]
            history.update({'weekday':rep})
            return history
        
    def _pros_week(self, expression, current, history):
        history = history.copy()
        year_adjust = 0
        W = current.isocalendar()[1] 
        reexp = r'y-.+W'
        extract = re.findall(reexp,expression)[0]

        # y-+1W
        if extract[2:-1] == '?':
            rep = str(W)
            history.update({'week':rep})
            return history
        elif extract[2] == '+':
            cur = W + int(extract[3:-1])
            if cur > 53:
                year_adjust = int(cur / 53)
                cur = cur % 53
            rep = str(cur)
            history.update({'year':year_adjust})
            history.update({'week':rep})
            return history
        elif extract[2] == '-':
            cur = W - int(extract[3:-1])
            if cur < 0:
                year_adjust = int(cur / 53) -1
                cur = cur % 53
            rep = str(cur)
            history.update({'year':year_adjust})
            history.update({'week':rep})
            return history
        else:
            rep = extract[2:-1]
            history.update({'week':rep})
            return history
    
    def create_from_D(self, history):
        if isinstance(history,dt.datetime):
            return history
        return dt.datetime(**history)
    
    def create_from_W(self, history):
        expression ='{}y-{}W-{}w-{}H:{}M:{}S'.format(history['year'],
                                                     history['week'],
                                                     history['weekday'],
                                                     history['hour'],
                                                     history['minute'],
                                                     history['second'])
        eval_time = dt.datetime.strptime(expression, "%Yy-%WW-%ww-%HH:%MM:%SS")
        return eval_time
    
    def ymd_reg(self,x):
        
        finds = list(set(re.findall(self.fix_ymd,x)) -set(['']))
        return finds

    def ymd_expression(self, result):
        def get_key(x):
            if x.isdigit():
                return x
            else:
                gets = self.time_dic.get(x)
                if x is None:
                    return '?'
                else:
                    return gets


        year_index = result.find('年')
        month_index = result.find('月')
        if result.find('日') != -1:
            date_index = result.find('日')
        else:
            date_index = result.find('号')

        if year_index != -1:
            year_key = result[0:year_index]  
            year = get_key(year_key)
        else:
            year = '?'
        if month_index != -1:
            month_key = result[year_index+1:month_index]
            month = get_key(month_key)
        else:
            month = '?'
        if date_index != -1:
            date_key = result[month_index+1:date_index]
            date = get_key(date_key)
        else:
            date = '?'
        formatted = '{}y-{}m-{}d-12H:00M:00S'.format(year, month, date)
        return formatted
    
    def evl_ymd(self,text):
        finds = self.ymd_reg(text)
        evls = []
        if len(finds) == 0:
            return evls
        else:
            for each in finds:
                evls.append({'pattern':each, 'expression':self.ymd_expression(each)})
        return evls
   

    def period_reg(self,x):
        
        finds = list(set(re.findall(self.fix_period,x)) - set(['']))
        return finds
    
    def period_expression(self,find):
        day_index = find.find('天')
        hour_index = find.find('小时')
        minute_index = find.find('分钟')
        if day_index != -1:
            day = find[0:day_index]
            if self.time_dic.get(day) is not None:
                day = int(self.time_dic.get(day))
            else:
                try:
                    day = int(day)
                except Exception:
                    day = 0
        else:
            day = 0

        if hour_index != -1:
            hour = find[day_index+1:hour_index]
            if self.time_dic.get(hour) is not None:
                hour = int(self.time_dic.get(hour))
            else:
                try:
                    hour = int(hour)
                except Exception:
                    hour = 0
        else:
            hour = 0

        if minute_index != -1:
            minute = find[hour_index+2:minute_index]
            if self.time_dic.get(minute) is not None:
                minute = int(self.time_dic.get(minute))
            else:
                try:
                    minute = int(minute)
                except Exception:
                    minute = 0
        else:
            minute = 0

        delta_minute = 24*60*day + 60*hour +minute 
        formatted = '?y-?m-?d-?H:+{}M:00S'.format(delta_minute)
        return formatted
    
    def evl_period(self,text):
        finds = self.period_reg(text)
        evls = []
        if len(finds) == 0:
            return evls
        else:
            for each in finds:
                evls.append({'pattern':each, 'expression':self.period_expression(each)})
        return evls



        
        
        
    def test_case1(self):
        """
        test if there is any overlab between self-defined and the fixed expression
        """
        error_result = []
        for each_pattern in self.serires.index.values:
            fixymd = self.evl_ymd(each_pattern)
            if len(fixymd) > 0:
                pattern = fixymd[0]['pattern']
#                 if pattern == each_pattern:
                error_result.append(each_pattern)
        print('============ test case 1 is below ==============')
        print(error_result)
        
    def test_case2(self):
        """
        This test is used to test all self define mapping;
        check is the evl string is correct
        """
        error_result = []
        for key in self.dict_ext:
            try:
                evl = self.evl(self.dict_ext[key])
            except Exception:
                error_result.append(key)
                print(key)
        print('============ test case 2 is below ==============')
        print(error_result )
        

In [2]:
t = TimePattern()

Time Zone is set from ENV: Asia/Shanghai


In [3]:
t.test_case2()

============ test case 2 is below ==============
[]


In [4]:
#after period 
s1 = '三天2小时5分钟后就还'
s2 = '四小时后就还'
s3 = '2小时就还'
s4 = '  哈哈水电费'

In [5]:
t.remove_time(s1)

'  后就还'

In [6]:
reg_num = r'(?:(?:一|二|两|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|二十|三十|四十|五十|六十|七十|八十|九十)|\d+)'
reg = r'(?:{}(?:天|日){}小时)|(?:{}(?:天|小时|日))'.format(reg_num,reg_num,reg_num)
finds = re.findall(reg,s1)


In [7]:
reg_num = r'(?:(?:一|二|两|三|四|五|六|七|八|九|十|十一|十二|十三|十四|十五|二十|三十|四十|五十|六十|七十|八十|九十)|\d+)'
reg = r'(?:{}(?:天|日))?(?:{}小时)?(?:{}分钟)?'.format(reg_num,reg_num,reg_num)
finds = list(set(re.findall(reg,s1)) - set(['']))

In [8]:
finds

['三天2小时5分钟']

In [9]:
mapping = {'今':'?','明':'+1','后':'+2','大后':'+3','下个':'+1','下下个':'+2',
                         '再下个':'+2','下下下个':'+3','后1个':'+1','后2个':'+2','后一个':'+1',
                         '后两个':'+2',
                         '一':'1','二':'2','三':'3','四':'4','五':'5',
                         '六':'6','七':'7','八':'8','九':'9','十':'10',
                         '十一':'11','十二':'12','十三':'13','十四':'14','十五':'15',
                         '十六':'16','十七':'17','十八':'18','十九':'19','二十':'20',
                         '二十一':'21','二十二':'22','二十三':'23','二十四':'24','二十五':'25',
                         '二十六':'26','二十七':'27','二十八':'28','二十九':'29','三十':'30','三十一':'31',
                         '四十':'40','五十':'50','六十':'60','七十':'70','八十':'80','九十':'90','一百':'100'}

In [10]:
find = '三天2小时5分钟'
day_index = find.find('天')
if day_index == -1:
    day_index = find.find('日')
hour_index = find.find('小时')
minute_index = find.find('分钟')
if day_index != -1:
    day = find[0:day_index]
    if mapping.get(day) is not None:
        day = int(mapping.get(day))
    else:
        try:
            day = int(day)
        except Exception:
            day = 0
else:
    day = 0
    
if hour_index != -1:
    hour = find[day_index+1:hour_index]
    if mapping.get(hour) is not None:
        hour = int(mapping.get(hour))
    else:
        try:
            hour = int(hour)
        except Exception:
            hour = 0
else:
    hour = 0
    
if minute_index != -1:
    minute = find[hour_index+2:minute_index]
    if mapping.get(minute) is not None:
        minute = int(mapping.get(minute))
    else:
        try:
            minute = int(minute)
        except Exception:
            minute = 0
else:
    minute = 0
    
delta_minute = 24*60*day + 60*hour +minute 
formatted = '?y-?m-?d-?H:+{}M:00S'.format(delta_minute)
    




In [139]:
formatted

'?y-?m-?d-?H:+4445M:00S'

In [140]:
t.evl(formatted)

datetime.datetime(2018, 8, 11, 15, 58, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>)

In [54]:
a = set(['1','2','3'])
a

{'1', '2', '3'}

In [55]:
b = '|'

In [56]:
b.join(a)

'1|2|3'

In [41]:
re.findall(r'','明天下午就还}')

['', '', '', '', '', '', '', '']

# 2018 0808 DEV

## case1 - 某年某月某日

In [166]:

num_cn = r'[一二三四五六七八九十零〇两百千0-9]'
rela_month = r'(?:下下下|下下|再下|下|(?:{}+))(?:个)?月'.format(num_cn)
fix_month = r'(?:{}+)月'.format(num_cn)
month_exp = r'(?:{}|{})'.format(rela_month,fix_month)


rela_date = '(?:今|明|后|大后|大大后|再后|(?:{})+)天'.format(num_cn)
date_descrip = r'(?:上旬|中旬|下旬|月初|月中|月末|初|中|底|末)'
# fix_date = r'(?:{}+)(?:日|号)'.format(num_cn)
fix_date = r'(?:(?:{}+)(?:日|号))|{}'.format(num_cn,date_descrip)
date_exp = r'(?:{}|{})'.format(rela_date,fix_date)

rela_year = r'(?:今|明|后|后后|再后|下一|再下一|下|再下|(?:{}))年'.format(num_cn)
fix_year = r'(?:{}+)年'.format(num_cn)
year_exp = r'(?:{}|{})'.format(rela_year,fix_year)

rela_hour= r'(?:{}+)小时'.format(num_cn)
hour_descrip = r'早晨|凌晨|早上|半夜|中午|下午|傍晚|晚上|清晨|午后'
minute_descrip = r'(?:一刻|半|过)|(?:{}+)'.format(num_cn)
fix_hour= r'(?:{})|(?:(?:{}+)点(?:{})*)'.format(hour_descrip,num_cn,minute_descrip)
hour_exp = r'(?:{}|{})'.format(rela_hour,fix_hour)



rela_week = r'(?:(?:这|下|再下|下下|再后)(?:{}+)?(?:个)?)|(?:{}+)'.format(num_cn,num_cn)
weekDay = r'[1-7天一二三四五六七]'
week_unit = r'周|星期|礼拜'
exp_week = r'(?:(?:{rw})?(?:{wu})+(?:{wd})?)+'.format(rw=rela_week,wu=week_unit,wd=weekDay)

exp_ymd = r'(?:(?:{y})|(?:{m})|(?:{d})|(?:{h}))+'.format(y=year_exp,m=month_exp,d=date_exp,h=hour_exp)


In [168]:
re.findall(exp_ymd,'我')

['今天']

In [226]:

finds1 = re.findall(exp_ymd,'我下个月10号可以还')[0]
finds2 = re.findall(exp_ymd,'我1月10号二零年可以还')[0]
finds2

'1月10号二零年'

### process year

In [297]:
#extract year

def YearExtractor(sentence):
    #########
    #######define
    
    def Year2Exp(text):
        """
        text has format of "<x>年"
        output: extracted expression
        """
        year_map = {'今':'?','明':'+1','后':'+2','后后':'+3',
                    '再后':'+3','下一':'+1','下':'+1','再下一':'+2','再下':'+2'}
        def CHN2Year(x):
            ##TODO: more case needs to be handled
            # case 1. digits
            if x.isdigit():
                # case 1.1 length is 4. eg 2017
                if len(x) == 4:
                    return x
                # case 1.2 length is not 4. which may mean calcuate delta
                else:
                    return '+'+x
            # needs to handle more cases
            else:
                return '?'
        index = text.find('年')
        y = None
        if index == -1:
            y = '?'
        else:
            context = text[:index]
            # step 1. get mapping from year_map
            if year_map.get(context) is not None:
                y = year_map.get(context)
            # step 2. CHN to canlender year    
            else:
                y = CHN2Year(context)
        return y
    ########################## end define ############################
    extracted_list = re.findall(year_exp,sentence)
    finds_list = []
    if len(extracted_list) > 0:
        for each in extracted_list:
            try:
                y = Year2Exp(each)
            except Exception:
                y = '?'
            finally:
                finds_list.append((each,y))
    else:
        y ='?'
        finds_list.append((None,y))
    return finds_list

# Process Month

In [231]:
finds2 = '下个月'
print(finds2)
print(re.findall(month_exp,finds2))
sentence = finds2
year_extracted_list = re.findall(month_exp,sentence)[0]

下个月
['下个月']


In [291]:
# Month2Exp
text = '五月'
c = CHN2NUM()
def MonthExtractor(sentence):
    """
    extract month info and convert to expression
    """
    def Month2Exp(text):
        month_map = {'下下下':'+3','下下':'+2','再下':'+2','下':'+1'}
        def CHN2Month(x,rela=True):
            """
            CHN2Month("二",rela=True)   --> +2
            CHN2Month("二",rela=False)   --> 2
            CHN2Month("二十",rela=False) --> +20
            """
            if x.isdigit():
                if int(x) > 12 or rela:
                    return '+'+x
                else:
                    return x
            # if not digit, then transform        
            else:
                try:
                    x = str(c.transform(x))
                    if int(x) > 12 or rela:
                        x = '+'+x                
                except Exception:
                    x = '?'
                finally:
                    return x
        index = text.find('月')
        m = None
        if index == -1:
            m = '?'
        else:
            # step 1. get mapping from month_map
            index_rala = text.find('个月')
            if index_rala != -1:
                index = index_rala
            content = text[:index]
            if month_map.get(content) is not None:
                m = month_map.get(content)

            # step 2. CHN to canlender month
            else:
                if index_rala != -1:
                    ## process relative month logic
                    m = CHN2Month(content,rela=True)
                else:
                    ## process nonrelative month logic
                    m = CHN2Month(content,rela=False)
        return m
    ################## end define ############
    extracted_list = re.findall(month_exp,sentence)
    finds_list = []
    if len(extracted_list) > 0:
        for each in extracted_list:
            try:
                m = Month2Exp(each)
            except Exception:
                m = '?'
            finally:
                finds_list.append((each,m))
    else:
        m = '?'
        finds_list.append((None,m))
    return finds_list

In [290]:
MonthExtractor('我1月就还')

[('1月', '1')]

In [281]:
a.find('路')

-1

In [269]:
m

'+20'

In [259]:

def CHN2Month(x,rela=True):
    """
    CHN2Month("二",rela=True)   --> +2
    CHN2Month("二",rela=False)   --> 2
    CHN2Month("二十",rela=False) --> +20
    """
    if x.isdigit():
        if int(x) > 12 or rela:
            return '+'+x
        else:
            return x
    # if not digit, then transform        
    else:
        try:
            x = str(c.transform(x))
            if int(x) > 12 or rela:
                x = '+'+x                
        except Exception:
            x = '?'
        finally:
            return x
  


In [263]:
CHN2Month('二',True)

'+2'

In [ ]:
rela_month = r'(?:下下下|下下|再下|下|(?:{}+))(?:个)?月'.format(num_cn)


In [3]:
class CHN2NUM:
    def __init__(self):
        self.CN_NUM = {
    '〇' : 0, '一' : 1, '二' : 2, '三' : 3, '四' : 4, '五' : 5, '六' : 6, '七' : 7, '八' : 8, '九' : 9, '零' : 0,
    '壹' : 1, '贰' : 2, '叁' : 3, '肆' : 4, '伍' : 5, '陆' : 6, '柒' : 7, '捌' : 8, '玖' : 9, '貮' : 2, '两' : 2
}

        for i in range(10):
            self.CN_NUM[str(i)] = i
        self.CN_UNIT = {
                            '十' : 10,
                            '拾' : 10,
                            '百' : 100,
                            '佰' : 100,
                            '千' : 1000,
                            '仟' : 1000,
                            '万' : 10000,
                            '萬' : 10000,
                            '亿' : 100000000,
                            '億' : 100000000,
                            '兆' : 1000000000000,
                        }

    def transform(self,cn:str):
        unit = 0   # current
        ldig = []  # digest
        for cndig in reversed(cn):
            if cndig in self.CN_UNIT:
                unit = self.CN_UNIT.get(cndig)
                if unit == 10000 or unit == 100000000:
                    ldig.append(unit)
                    unit = 1
            else:
                dig = self.CN_NUM.get(cndig)
                if unit:
                    dig *= unit
                    unit = 0
                ldig.append(dig)
        if unit == 10:
            ldig.append(10)
        val, tmp = 0, 0
        for x in reversed(ldig):
            if x == 10000 or x == 100000000:
                val += tmp * x
                tmp = 0
            else:
                tmp += x
        val += tmp
        return val


class ReExtractor:
    def __init__(self):
        self.CN2NUM = CHN2NUM()
        self.num_cn = r'[一二三四五六七八九十零〇两百千0-9]'
        
        self.year_rela_exp = r'(?:今|明|后|后后|再后|下一|再下一|下|再下|(?:{}))年'.format(self.num_cn)
        self.year_fix_exp = r'(?:{}+)年'.format(self.num_cn)
        self.year_exp = r'(?:{}|{})'.format(self.year_rela_exp,self.year_fix_exp)
        
        self.month_rela_exp = r'(?:下下下|下下|再下|下|(?:{}+))(?:个)?月'.format(self.num_cn)
        self.month_fix_exp = r'(?:{}+)月'.format(self.num_cn)
        self.month_exp = r'(?:{}|{})'.format(self.month_rela_exp,self.month_fix_exp)


        self.date_rela_exp = '(?:今|明|后|大后|大大后|再后|(?:{})+)天'.format(self.num_cn)
        self.date_descrip_exp = r'(?:上旬|中旬|下旬|月初|月中|月末|初|中|底|末)'
        self.date_fix_exp = r'(?:(?:{}+)(?:日|号))|{}'.format(self.num_cn,self.date_descrip_exp)
        self.date_exp = r'(?:{}|{})'.format(self.date_rela_exp,self.date_fix_exp)

        

        self.hour_rela_exp= r'(?:{}+)小时'.format(self.num_cn)
        self.hour_descrip_exp = r'早晨|凌晨|早上|半夜|中午|下午|傍晚|晚上|清晨|午后'
        self.minute_descrip_exp = r'(?:一刻|半|过)|(?:{}+)'.format(self.num_cn)
        self.hour_fix_exp= r'(?:{})|(?:(?:{}+)点(?:{})*)'.format(self.hour_descrip_exp,
                                                                   self.num_cn,
                                                                   self.minute_descrip_exp)
        self.hour_exp = r'(?:{}|{})'.format(self.hour_rela_exp,self.hour_fix_exp)



        self.week_rela_exp = r'(?:(?:这|下|再下|下下|再后)(?:{}+)?(?:个)?)|(?:{}+)'.format(self.num_cn,self.num_cn)
        self.weekDay_exp = r'[1-7天一二三四五六七]'
        self.week_unit = r'周|星期|礼拜'
        self.exp_week = r'(?:(?:{rw})?(?:{wu})+(?:{wd})?)+'.format(rw=self.week_rela_exp,
                                                                   wu=self.week_unit,
                                                                   wd=self.weekDay_exp)

        self.exp_ymd = r'(?:(?:{y})|(?:{m})|(?:{d})|(?:{h}))+'.format(y=self.year_exp,
                                                                      m=self.month_exp,
                                                                      d=self.date_exp,
                                                                      h=self.hour_exp)
        
    def YearExtractor(self,sentence):
        """
        this function will return a list of year it can extract
        """
        #########
        #######define
        
        def Year2Exp(text):
            """
            text has format of "<x>年"
            output: extracted expression
            """
            year_map = {'今':'?','明':'+1','后':'+2','后后':'+3',
                        '再后':'+3','下一':'+1','下':'+1','再下一':'+2','再下':'+2'}
            def CHN2Year(x):
                ##TODO: more case needs to be handled
                # case 1. digits
                if x.isdigit():
                    # case 1.1 length is 4. eg 2017
                    if len(x) == 4:
                        return x
                    # case 1.2 length is not 4. which may mean calcuate delta
                    else:
                        return '+'+x
                # needs to handle more cases
                else:
                    return '?'
            index = text.find('年')
            y = None
            if index == -1:
                y = '?'
            else:
                context = text[:index]
                # step 1. get mapping from year_map
                if year_map.get(context) is not None:
                    y = year_map.get(context)
                # step 2. CHN to canlender year    
                else:
                    y = CHN2Year(context)
            return y
        ########################## end define ############################
        extracted_list = re.findall(self.year_exp,sentence)
        finds_list = []
        if len(extracted_list) > 0:
            for each in extracted_list:
                try:
                    y = Year2Exp(each)
                except Exception:
                    y = '?'
                finally:
                    finds_list.append((each,y))
        else:
            y = '?'
            finds_list.append((None,y))
        return finds_list 
    
    def MonthExtractor(self,sentence):
        """
        extract month info and convert to expression
        """
        def Month2Exp(text):
            month_map = {'下下下':'+3','下下':'+2','再下':'+2','下':'+1'}
            def CHN2Month(x,rela=True):
                """
                CHN2Month("二",rela=True)   --> +2
                CHN2Month("二",rela=False)   --> 2
                CHN2Month("二十",rela=False) --> +20
                """
                if x.isdigit():
                    if int(x) > 12 or rela:
                        return '+'+x
                    else:
                        return x
                # if not digit, then transform        
                else:
                    try:
                        x = str(self.CHN2NUM.transform(x))
                        if int(x) > 12 or rela:
                            x = '+'+x                
                    except Exception:
                        x = '?'
                    finally:
                        return x
            index = text.find('月')
            m = None
            if index == -1:
                m = '?'
            else:
                # step 1. get mapping from month_map
                index_rala = text.find('个月')
                if index_rala != -1:
                    index = index_rala
                content = text[:index]
                if month_map.get(content) is not None:
                    m = month_map.get(content)

                # step 2. CHN to canlender month
                else:
                    if index_rala != -1:
                        ## process relative month logic
                        m = CHN2Month(content,rela=True)
                    else:
                        ## process nonrelative month logic
                        m = CHN2Month(content,rela=False)
            return m
        ################## end define ############
        extracted_list = re.findall(self.month_exp,sentence)
        finds_list = []
        if len(extracted_list) > 0:
            for each in extracted_list:
                try:
                    m = Month2Exp(each)
                except Exception:
                    m = '?'
                finally:
                    finds_list.append((each,m))
        else:
            m = '?'
            finds_list.append((None,m))
        return finds_list
        

In [4]:
r = ReExtractor()

In [5]:

r.YearExtractor('2年以后')

[('2年', '+2')]

In [8]:
%%timeit
r.MonthExtractor('2年2个月以后')

3 µs ± 65.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
